# ---- 1. Scraping Dataset ----

In [7]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# --- Pengaturan Awal ---
BASE_URL = "https://www.masakapahariini.com/resep/masakan-tradisional/"
TOTAL_PAGES = 62
all_recipe_links = []

# Mengatur dan memulai browser Chrome secara otomatis
try:
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    print("Memulai proses scraping...")

    # --- Looping Melalui Setiap Halaman ---
    for page_num in range(1, TOTAL_PAGES + 1):
        if page_num == 1:
            page_url = BASE_URL
        else:
            page_url = f"{BASE_URL}page/{page_num}/"

        print(f"Mengambil data dari halaman: {page_url}")
        driver.get(page_url)
        time.sleep(3) # Memberi waktu halaman untuk memuat

        # --- Mencari Elemen Link Resep ---
        # Selector ini menargetkan SEMUA link resep di halaman, bukan hanya yang pertama.
        # Ini adalah cara yang benar untuk mengambil semua data yang kita inginkan.
        recipe_elements = driver.find_elements(By.CSS_SELECTOR, "h3.card-title a")

        if not recipe_elements:
             print(f"Peringatan: Tidak ada resep yang ditemukan di halaman {page_num}. Mungkin struktur halaman berubah.")

        # Mengambil atribut 'href' (link) dari setiap elemen yang ditemukan
        for element in recipe_elements:
            link = element.get_attribute('href')
            if link:
                all_recipe_links.append(link)

    print(f"\nProses scraping selesai. Total link yang ditemukan: {len(all_recipe_links)}")

finally:
    if 'driver' in locals():
        driver.quit()

# --- Menampilkan Hasil ---
print("\n--- Daftar Semua Link Resep ---")
for link in all_recipe_links:
    print(link)

Memulai proses scraping...
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/2/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/3/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/4/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/5/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/6/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/7/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/8/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/9/
Mengambil data dari halaman: https://www.masakapahariini.com/resep/masakan-tradisional/page/10/
Mengambil data dari halaman: 

In [10]:
# Simpan semua link resep ke file teks
with open("all_recipe_links.txt", "w", encoding="utf-8") as f:
    for link in all_recipe_links:
        f.write(link + "\n")
print("Daftar link resep berhasil disimpan ke 'all_recipe_links.txt'.")

Daftar link resep berhasil disimpan ke 'all_recipe_links.txt'.


In [2]:
# Membaca file all_recipe_links.txt dan mengubahnya menjadi list
with open("all_recipe_links.txt", "r", encoding="utf-8") as f:
    all_recipe_links = [line.strip() for line in f if line.strip()]
print(f"Total link yang dibaca: {len(all_recipe_links)}")

Total link yang dibaca: 920


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv # Modul untuk bekerja dengan file CSV
import os # Modul untuk memeriksa keberadaan file

# URL dari halaman detail resep yang ingin di-scrape
# GANTI URL INI DENGAN URL RESEP YANG SEBENARNYA.
# Untuk pengujian, Anda bisa membuat daftar URL dan melakukan loop
urls_to_scrape = all_recipe_links 

# Nama file CSV untuk menyimpan hasil
csv_filename = "data_resep_new.csv"
# Header untuk file CSV
csv_header = ["Judul Masakan", "Waktu Masak", "Tingkat Kesulitan", "Jumlah Porsi", "Bahan-bahan", "URL Sumber"]

# Setup WebDriver (menggunakan ChromeDriverManager untuk kemudahan)
driver = None 
try:
    print("Sedang menyiapkan WebDriver...")
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--no-sandbox") 
    chrome_options.add_argument("--disable-dev-shm-usage") 
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")

    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Memeriksa apakah file CSV sudah ada untuk menulis header
    file_exists = os.path.isfile(csv_filename)
    
    # Membuka file CSV dalam mode 'append' (a+)
    with open(csv_filename, mode='a+', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        
        # Menulis header hanya jika file baru dibuat
        if not file_exists or os.path.getsize(csv_filename) == 0:
            writer.writerow(csv_header)
            print(f"File CSV '{csv_filename}' dibuat dengan header.")

        # Loop melalui setiap URL untuk di-scrape
        for url in urls_to_scrape:
            print(f"\nMengakses URL: {url}")
            driver.get(url)
            time.sleep(5) 

            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            # Pesan sederhana bahwa parsing berhasil, detail tidak ditampilkan lagi di sini
            # print("Konten halaman berhasil diambil dan di-parse.") 

            # --- 1. Mengambil Judul Masakan ---
            title_element = soup.find('h1', class_='title') 
            if not title_element:
                title_element = soup.find('h1') 
            judul_masakan = title_element.get_text(strip=True) if title_element else "Judul tidak ditemukan"

            # --- 2. Mengambil Waktu Masak & 3. Tingkat Kesulitan ---
            waktu_masak = "Waktu tidak ditemukan"
            tingkat_kesulitan = "Kesulitan tidak ditemukan"

            recipe_features_div = soup.find('div', class_='_recipe-features')
            if recipe_features_div:
                time_link = recipe_features_div.find('a', class_=lambda x: x and 'btn' in x and 'item' in x)
                if time_link:
                    time_span = time_link.find('span')
                    if time_span and ('jam' in time_span.text.lower() or 'mnt' in time_span.text.lower() or 'menit' in time_span.text.lower()):
                         waktu_masak = time_span.get_text(strip=True)

                difficulty_link = recipe_features_div.find('a', class_='icon_difficulty')
                if difficulty_link:
                    all_text_in_difficulty_link = difficulty_link.get_text(strip=True)
                    if all_text_in_difficulty_link:
                         tingkat_kesulitan = all_text_in_difficulty_link
                else: 
                    possible_difficulty_texts = ['mudah', 'sedang', 'sulit']
                    difficulty_links_all = recipe_features_div.find_all('a', class_=lambda x: x and 'btn' in x and 'item' in x)
                    for link_dl in difficulty_links_all:
                        link_text_lower_dl = link_dl.get_text(strip=True).lower()
                        if any(difficulty_text in link_text_lower_dl for difficulty_text in possible_difficulty_texts) and link_text_lower_dl != waktu_masak.lower():
                            tingkat_kesulitan = link_dl.get_text(strip=True)
                            break
            
            # --- 4. Mengambil Jumlah Porsi ---
            jumlah_porsi = "Jumlah porsi tidak ditemukan"
            portions_div = soup.find('div', class_='portions')
            if portions_div:
                portions_value_div = portions_div.find('div', id='portions-value')
                portions_label_div = portions_div.find('div', class_='portions-label')
                if portions_value_div and portions_label_div:
                    jumlah_porsi = f"{portions_value_div.get_text(strip=True)} {portions_label_div.get_text(strip=True)}"
                elif portions_value_div: 
                     jumlah_porsi = f"{portions_value_div.get_text(strip=True)} Porsi"

            # --- 5. Mengambil Bahan-Bahan ---
            bahan_list_raw = [] 
            recipe_ingredients_div = soup.find('div', class_='_recipe-ingredients')
            if recipe_ingredients_div:
                elements = recipe_ingredients_div.find_all(['p', 'div'], recursive=False) 
                current_section_title = "Bahan Utama" 

                for el in elements:
                    if el.name == 'p' and 'recipe-ingredients-title' in el.get('class', []):
                        current_section_title = el.get_text(strip=True)
                        # Judul bagian tidak ditambahkan ke bahan_list_raw untuk format paragraf
                        # if not bahan_list_raw or not bahan_list_raw[-1].endswith(f"--- {current_section_title} ---"):
                        #     bahan_list_raw.append(f"\n--- {current_section_title} ---")
                    elif el.name == 'div' and ('d-flex' in el.get('class', []) or el.find('div', class_='item')): 
                        part_div = el.find('div', class_='part')
                        item_div = el.find('div', class_='item')

                        if item_div: 
                            quantity = part_div.get_text(strip=True) if part_div else ""
                            name = item_div.get_text(strip=True)
                            if name: 
                                bahan_list_raw.append(f"{quantity} {name}".strip())
            
            # Menggabungkan semua bahan menjadi satu string paragraf untuk CSV
            # Item bahan yang valid (bukan string kosong) akan digabungkan dengan ", "
            bahan_paragraf_for_csv = ", ".join([bahan for bahan in bahan_list_raw if bahan.strip()])


            # --- Menampilkan Hasil ke Konsol (Disederhanakan) ---
            # Tidak menampilkan detail lagi, hanya pesan sukses per resep
            # print("\n--- Hasil Scrapping Data Resep (Konsol) ---")
            # print(f"Judul Masakan    : {judul_masakan}")
            # print(f"Waktu Masak      : {waktu_masak}")
            # print(f"Tingkat Kesulitan: {tingkat_kesulitan}")
            # print(f"Jumlah Porsi     : {jumlah_porsi}")
            # print(f"URL Sumber       : {url}")

            # if bahan_list_raw: # Menggunakan bahan_list_raw untuk pengecekan sebelum join
            #     print("\nBahan-bahan (untuk CSV akan jadi paragraf):")
            #     for bahan_item in bahan_list_raw: # Loop melalui bahan mentah untuk tampilan konsol jika diperlukan
            #         if bahan_item.strip() and not bahan_item.strip().startswith("---"):
            #             print(f"- {bahan_item.strip()}")
            # else:
            #     print("\nBahan-bahan: Tidak ditemukan")

            # --- Menyimpan data ke file CSV ---
            row_data = [judul_masakan, waktu_masak, tingkat_kesulitan, jumlah_porsi, bahan_paragraf_for_csv, url]
            writer.writerow(row_data)
            # Pesan sukses yang lebih sederhana
            print(f"Data untuk '{judul_masakan}' berhasil diambil dan disimpan ke '{csv_filename}'.")


except Exception as e:
    print(f"Terjadi kesalahan: {e}")
    import traceback
    traceback.print_exc()

finally:
    if driver:
        print("\nMenutup WebDriver.")
        driver.quit()

print(f"\nProses scraping selesai. Data tersimpan di '{csv_filename}'.")


Sedang menyiapkan WebDriver...
File CSV 'data_resep_new.csv' dibuat dengan header.

Mengakses URL: https://www.masakapahariini.com/resep/resep-tempe-goreng-tepung-pedas/
Data untuk 'Resep Tempe Goreng Tepung Pedas Krispi yang Bikin Nagih' berhasil diambil dan disimpan ke 'data_resep_new.csv'.

Mengakses URL: https://www.masakapahariini.com/resep/resep-kikil-sapi-kuah-pedas-manis-empuk-kenyal-dan-sedap/
Data untuk 'Resep Kikil Sapi Kuah Pedas Manis, Empuk, Kenyal, dan Sedap' berhasil diambil dan disimpan ke 'data_resep_new.csv'.

Mengakses URL: https://www.masakapahariini.com/resep/resep-tempe-penyet-saus-petis/
Data untuk 'Resep Tempe Penyet Sambal Petis Sederhana ala Rumahan' berhasil diambil dan disimpan ke 'data_resep_new.csv'.

Mengakses URL: https://www.masakapahariini.com/resep/resep-bakmi-jogja-goreng/
Data untuk 'Resep Bakmi Jawa Jogja Goreng Enak dan Gampang Bikinnya' berhasil diambil dan disimpan ke 'data_resep_new.csv'.

Mengakses URL: https://www.masakapahariini.com/resep/r

# ---- 2. Melihat Bahan yang Sering Muncul ----

In [1]:
import pandas as pd
import re
from collections import Counter

# Membaca data dari file CSV yang sudah ada
df = pd.read_csv('data_resep_new.csv', encoding='utf-8')

# Fungsi untuk membersihkan string: menghapus angka dan simbol, hanya menyisakan huruf dan spasi
def clean_text(text):
    # Hapus angka dan simbol, hanya sisakan huruf dan spasi
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    # Ubah ke huruf kecil dan hapus spasi berlebih
    text = text.lower().strip()
    return text

# Bersihkan kolom "Bahan-bahan" dan pecah menjadi list kata
df['Bahan-bahan_cleaned'] = df['Bahan-bahan'].apply(lambda x: clean_text(x).split())

# Gabungkan semua list kata dari seluruh baris menjadi satu list besar
all_words = [word for words_list in df['Bahan-bahan_cleaned'] for word in words_list]

# Hitung frekuensi kemunculan setiap kata
word_counts = Counter(all_words)

# Tentukan top-n kata yang paling sering muncul (misal n=10)
n = 150
top_n_words = word_counts.most_common(n)

# Tampilkan hasil
print(f"Top {n} kata yang paling sering muncul di kolom 'Bahan-bahan':")
for word, count in top_n_words:
    print(f"{word}: {count}")

Top 150 kata yang paling sering muncul di kolom 'Bahan-bahan':
bawang: 1611
sdt: 1439
g: 1416
merah: 1371
sdm: 1300
buah: 1167
putih: 1063
butir: 1062
cm: 929
daun: 895
air: 877
potong: 694
iris: 690
siung: 685
ml: 684
ayam: 670
cabai: 663
minyak: 635
garam: 622
lembar: 608
manis: 592
untuk: 556
kaldu: 536
kecap: 533
batang: 513
bubuk: 500
memarkan: 444
sdmbango: 421
jeruk: 394
sdtroyco: 366
gula: 357
goreng: 347
merica: 331
sapi: 323
rawit: 318
menumis: 311
tipis: 294
salam: 286
serai: 280
jahe: 266
lengkuas: 263
daging: 228
kemiri: 223
tepung: 208
ketumbar: 208
bagian: 208
santan: 206
tomat: 205
telur: 202
kunyit: 197
sangrai: 197
pasir: 187
nan: 187
l: 178
dadu: 174
cincang: 166
halus: 165
rebus: 144
asam: 133
buang: 132
nipis: 130
menggoreng: 127
kupas: 124
hijau: 120
potongpotong: 120
jawa: 118
sdmroyco: 118
dan: 116
besar: 106
kacang: 104
ekor: 104
ikan: 104
keriting: 102
haluskan: 101
serong: 99
merebus: 95
pala: 93
kasar: 91
kotak: 90
panas: 82
sayur: 81
kelapa: 75
kental: 75
t

sdt, g, buah, cm, air, potong, iris, siung, ml, lembar, untuk, batang, bubuk, memarkan, menumis, tipis, bagian, sangrai, nan, dadu, cincang, halus, rebus, menggoreng, kupas, potongpotong, dan, besar, ekor, haluskan, serong, merebus, kasar, kotak, panas, kental, dengan, bakar, hingga, belah, larutkan, parut, instan, ambil, rendah, butiran, biji, kecil, tiriskan, sisir, tanah, siap, dalam, cuci, has

In [11]:
# Daftar kata yang ingin dihapus
stopwords = [
    'sdt', 'g', 'buah', 'cm', 'air', 'potong', 'iris', 'siung', 'ml', 'lembar', 'untuk', 'batang', 'bubuk',
    'memarkan', 'menumis', 'tipis', 'bagian', 'sangrai', 'nan', 'dadu', 'cincang', 'halus', 'rebus', 'menggoreng',
    'kupas', 'potongpotong', 'dan', 'besar', 'ekor', 'haluskan', 'serong', 'merebus', 'kasar', 'kotak', 'panas',
    'kental', 'dengan', 'bakar', 'hingga', 'belah', 'larutkan', 'parut', 'instan', 'ambil', 'rendah', 'butiran',
    'biji', 'kecil', 'tiriskan', 'sisir', 'tanah', 'siap', 'dalam', 'cuci', 'has', 'sdm', 'butir', 'sdmbango', 'sdtroyco',
    'l', 'buang', 'sdmroyco', 'bumbu', 'atau', 'matang', 'rendam', 'seduh', 'siangi', 'sengkel', 'korek', 'ruas',
    'api', 'kocok', 'memanjang', 'bungkusroyco', 'bombay', 'fillet', 'bersih', 'muda', 'light', 'ukuran', 'kg',
    'lepas', 'panjang', 'tusuk', 'pedas', 'encer',  'kampung', 'dari', 'hangat', 'giling', 'papan', 'cair', 'dua',
    'tanpa', 'ikat', 'hitam', 'pakai', 'airnya', 'mlbango', 'es', 'porsi', 'utuh', 'kulitnya', 'khas', 'petiki',
    'powder', 'putihnya', 'setengah', 'blansir', 'gram', 'secukupnya', 'membungkus', 'bungkusbango', 'simpul',
    'tebal', 'filet', 'bersihkan', 
]

# Hapus kata-kata tersebut dari kolom 'Bahan-bahan_cleaned'
df['Bahan-bahan_cleaned_nostop'] = df['Bahan-bahan_cleaned'].apply(
    lambda words: [w for w in words if w not in stopwords]
)

In [12]:
import pandas as pd
import re
from collections import Counter

# Gabungkan semua list kata dari seluruh baris menjadi satu list besar
all_words = [word for words_list in df['Bahan-bahan_cleaned_nostop'] for word in words_list]

# Hitung frekuensi kemunculan setiap kata
word_counts = Counter(all_words)

# Tentukan top-n kata yang paling sering muncul (misal n=10)
n = 200
top_n_words = word_counts.most_common(n)

# Tampilkan hasil
print(f"Top {n} kata yang paling sering muncul di kolom 'Bahan-bahan':")
for word, count in top_n_words:
    print(f"{word}: {count}")

Top 200 kata yang paling sering muncul di kolom 'Bahan-bahan':
bawang: 1611
merah: 1371
putih: 1063
daun: 895
ayam: 670
cabai: 663
minyak: 635
garam: 622
manis: 592
kaldu: 536
kecap: 533
jeruk: 394
gula: 357
goreng: 347
merica: 331
sapi: 323
rawit: 318
salam: 286
serai: 280
jahe: 266
lengkuas: 263
daging: 228
kemiri: 223
tepung: 208
ketumbar: 208
santan: 206
tomat: 205
telur: 202
kunyit: 197
pasir: 187
asam: 133
nipis: 130
hijau: 120
jawa: 118
kacang: 104
ikan: 104
keriting: 102
pala: 93
sayur: 81
kelapa: 75
tahu: 75
terigu: 72
seledri: 68
beras: 68
tempe: 65
kol: 65
kayu: 65
jintan: 65
susu: 62
wortel: 62
kambing: 62
kulit: 62
jamur: 61
kemangi: 59
taoge: 57
lemak: 55
bango: 53
tulang: 53
kencur: 53
nasi: 52
udang: 50
cengkih: 50
terasi: 49
kentang: 49
sambal: 48
tapioka: 46
margarin: 46
mentimun: 42
dada: 40
limau: 34
daunnya: 34
kapulaga: 34
pandan: 33
asin: 32
pisang: 32
kerupuk: 30
lada: 30
saus: 28
kuning: 28
iga: 27
maizena: 25
labu: 25
siam: 23
mie: 22
lontong: 20
teri: 20
teng

bawang, cabai, gula, garam, merica, salam, serai, jahe, lengkuas, kemiri, ketumbar, santan, tomat, telur, kunyit, kacang, ikan, seledri, tahu, tempe, kol, wortel, , jamur, kemangi, kencur, cengkih, kemangi

In [16]:
from collections import Counter
from nltk.util import ngrams

# Pastikan kolom 'Bahan-bahan_cleaned' sudah berupa list kata per baris
# Gabungkan semua list kata dari seluruh baris menjadi satu list besar
all_words = [word for words_list in df['Bahan-bahan_cleaned_nostop'] for word in words_list]

# Fungsi untuk mencari n-gram paling sering muncul
def top_ngrams(words, n=2, top_k=20):
    ngram_list = list(ngrams(words, n))
    ngram_counts = Counter(ngram_list)
    return ngram_counts.most_common(top_k)

# Contoh: mencari 2-gram (bigram) dan 3-gram (trigram) paling sering muncul
top_bigrams = top_ngrams(all_words, n=2, top_k=150)
top_trigrams = top_ngrams(all_words, n=3, top_k=1)

print("Top 100 Bigram:")
for ngram, count in top_bigrams:
    print(f"{' '.join(ngram)}: {count}")

print("\nTop 1 Trigram:")
for ngram, count in top_trigrams:
    print(f"{' '.join(ngram)}: {count}")

Top 100 Bigram:
bawang putih: 684
bawang merah: 657
kecap manis: 511
merah bawang: 485
kaldu ayam: 356
cabai merah: 315
cabai rawit: 294
daun salam: 286
merica putih: 263
daun jeruk: 225
rawit merah: 217
gula pasir: 187
minyak bawang: 183
daun bawang: 165
merah cabai: 157
kaldu sapi: 133
putih bawang: 132
putih cabai: 131
gula merah: 130
jeruk nipis: 130
daging sapi: 120
garam merica: 111
asam jawa: 111
putih kemiri: 104
garam gula: 94
telur ayam: 94
serai daun: 92
merah goreng: 89
manis kaldu: 83
merah keriting: 82
ayam minyak: 77
minyak sayur: 76
manis garam: 72
tepung terigu: 71
salam serai: 67
putih jahe: 67
kayu manis: 65
manis minyak: 60
putih garam: 60
ayam daun: 59
minyak cabai: 59
salam lengkuas: 56
ayam bawang: 52
lengkuas daun: 49
daun kemangi: 49
susu lemak: 49
salam daun: 47
ketumbar garam: 47
bango kecap: 47
merah daun: 47
goreng bawang: 46
tepung tapioka: 46
merah kemiri: 46
minyak goreng: 45
putih ketumbar: 44
bawang goreng: 43
garam minyak: 43
putih minyak: 43
lengkuas

# ---- 3. Menghitung Kemunculan Bahan-Bahan Fix dan Mengubah Format Kolom Bahan-Bahan ----

[bawang putih, bawang merah, kecap manis, kaldu ayam, cabai merah, cabai rawit, daun salam, daun jeruk, gula pasir, daun bawang, kaldu sapi, daging ayam, daging sapi, gula merah, jeruk nipis, asam jawa, kayu manis, cabai hijau, daging kambing, limau, bawang, cabai, gula, garam, merica, salam, serai, jahe, lengkuas, kemiri, ketumbar, santan, tomat, telur, kunyit, kacang, ikan, seledri, tahu, tempe, kol, wortel, , jamur, kemangi, kencur, cengkih, jagung, cuka, wijen, udang]

In [18]:
from collections import Counter

# Daftar frasa/kata target (urutkan dari frasa terpanjang ke terpendek untuk mencegah overlap)
target_list = [
    'bawang putih', 'bawang merah', 'kecap manis', 'kaldu ayam', 'cabai merah', 'cabai rawit', 'daun salam', 'daun jeruk',
    'gula pasir', 'daun bawang', 'kaldu sapi', 'daging ayam', 'daging sapi', 'gula merah', 'jeruk nipis', 'asam jawa',
    'kayu manis', 'cabai hijau', 'daging kambing', 'limau', 'bawang', 'cabai', 'gula', 'garam', 'merica', 'salam',
    'serai', 'jahe', 'lengkuas', 'kemiri', 'ketumbar', 'santan', 'tomat', 'telur', 'kunyit', 'kacang', 'ikan',
    'seledri', 'tahu', 'tempe', 'kol', 'wortel', '', 'jamur', 'kemangi', 'kencur', 'cengkih', 'jagung', 'cuka', 'wijen', 'udang'
]

# Pisahkan frasa (mengandung spasi) dan kata satuan
target_phrases = [item for item in target_list if ' ' in item and item.strip()]
target_words = [item for item in target_list if ' ' not in item and item.strip()]
# Gabungkan dan urutkan: frasa dulu, lalu kata
all_targets = target_phrases + target_words

def extract_bahan_fix(words):
    # Gabungkan list kata menjadi string untuk pencarian frasa
    text = ' '.join(words)
    found = []
    used_idx = set()
    # Cek frasa dulu
    for phrase in target_phrases:
        # Cari frasa secara eksak
        pattern = r'\b' + re.escape(phrase) + r'\b'
        match = re.search(pattern, text)
        if match:
            found.append(phrase)
            # Hapus frasa dari text agar tidak double count
            text = re.sub(pattern, '', text)
    # Sisa kata split lagi
    sisa_kata = text.split()
    # Cek kata satuan
    for word in target_words:
        if word in sisa_kata:
            found.append(word)
            # Hapus semua kemunculan kata ini agar tidak double count
            sisa_kata = [w for w in sisa_kata if w != word]
    return found

# Terapkan ke dataframe
df['bahan-bahan_fix'] = df['Bahan-bahan_cleaned_nostop'].apply(extract_bahan_fix)

# Jika ingin menambahkan count kemunculan per frasa/kata:
all_found = [item for sublist in df['bahan-bahan_fix'] for item in sublist]
bahan_fix_counts = Counter(all_found)
print("Jumlah kemunculan masing-masing bahan fix:")
for bahan, cnt in bahan_fix_counts.items():
    print(f"{bahan}: {cnt}")

Jumlah kemunculan masing-masing bahan fix:
bawang putih: 604
bawang merah: 561
kaldu ayam: 320
cabai rawit: 270
daun bawang: 159
merica: 308
ketumbar: 207
tempe: 61
kecap manis: 471
cabai merah: 282
daun salam: 275
daun jeruk: 221
kaldu sapi: 124
limau: 32
bawang: 97
garam: 534
serai: 271
jahe: 256
kemiri: 222
tomat: 197
kunyit: 171
gula pasir: 176
santan: 185
telur: 172
gula merah: 128
asam jawa: 108
lengkuas: 256
kacang: 90
jagung: 17
kemangi: 58
daging sapi: 114
seledri: 65
tahu: 70
kol: 64
wortel: 59
udang: 48
jeruk nipis: 123
daging kambing: 31
cabai hijau: 32
ikan: 98
jamur: 52
kayu manis: 65
cengkih: 50
daging ayam: 39
kencur: 53
wijen: 16
gula: 36
cuka: 15
cabai: 15


In [19]:
df['bahan-bahan_fix'] = df['bahan-bahan_fix'].apply(lambda x: ', '.join(x))
df.head()

,Judul Masakan,Waktu Masak,Tingkat Kesulitan,Jumlah Porsi,Bahan-bahan,URL Sumber,Bahan-bahan_cleaned,Bahan-bahan_cleaned_nostop,bahan-bahan_fix
0,Resep Tempe Goreng Tepung Pedas Krispi yang Bi...,30mnt,Mudah,4 Porsi,"300 g tempe, iris tipis, 150 g tepung terigu, ...",https://www.masakapahariini.com/resep/resep-te...,"[g, tempe, iris, tipis, g, tepung, terigu, sdm...","[tempe, tepung, terigu, tepung, maizena, bakin...","bawang putih, bawang merah, kaldu ayam, cabai ..."
1,"Resep Kikil Sapi Kuah Pedas Manis, Empuk, Keny...",2j 30mnt,Sedang,4 Porsi,500 g \t\t\t\ttunjang atau kikil y...,https://www.masakapahariini.com/resep/resep-ki...,"[g, tunjang, atau, kikil, yang, sudah, dibersi...","[tunjang, kikil, yang, sudah, dibersihkan, dau...","bawang putih, bawang merah, kecap manis, cabai..."
2,Resep Tempe Penyet Sambal Petis Sederhana ala ...,30mnt,Mudah,4 Porsi,"300 g tempe, potong tebal 2 cm, ½ sdm garam, 1...",https://www.masakapahariini.com/resep/resep-te...,"[g, tempe, potong, tebal, cm, sdm, garam, sdt,...","[tempe, garam, ketumbar, santan, kelapa, cabai...","bawang putih, kecap manis, cabai merah, cabai ..."
3,Resep Bakmi Jawa Jogja Goreng Enak dan Gampang...,1jam,Mudah,2 Porsi,"200 g mie basah, 150 g dada ayam fillet, poton...",https://www.masakapahariini.com/resep/resep-ba...,"[g, mie, basah, g, dada, ayam, fillet, potong,...","[mie, basah, dada, ayam, telur, ayam, sawi, hi...","bawang putih, bawang merah, kecap manis, kaldu..."
4,"Resep Sayur Asem Bening Sederhana ala Rumahan,...",45mnt,Mudah,4 Porsi,"1 buah \t\t\t\tjagung manis, poton...",https://www.masakapahariini.com/resep/resep-sa...,"[buah, jagung, manis, potong, bagian, buah, la...","[jagung, manis, labu, siam, melinjo, kacang, k...","bawang merah, kaldu ayam, cabai merah, daun sa..."


# ---- 4. Menyimpan Dataset Baru ----

In [20]:
df.to_csv('data_resep_new_cleaned.csv', index=False, encoding='utf-8')